In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn import manifold
import matplotlib.pyplot as plt
import seaborn as sns
import PIL

In [2]:
latent_dim = 8
sigma_coeff = 1.0
lambda_coeff = 0.01
start_learning_rate = 0.0002
epochs_num = 50
batch_size = 100
dim_h = 32

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
torch.manual_seed(30)
if use_cuda:
  torch.cuda.manual_seed(30)

In [4]:
trainset = datasets.MNIST(root='./MNIST/', train=True, transform=transforms.ToTensor(), download=True)
testset  = datasets.MNIST(root='./MNIST/', train=False, transform=transforms.ToTensor(), download=True)

In [5]:
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False)

### Encoder model

In [6]:
class WAE_ENCODER(nn.Module):
    def __init__(self):
        super(WAE_ENCODER, self).__init__()
        
        self.conv_part = nn.Sequential(
            nn.Conv2d(1, dim_h, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(dim_h, dim_h * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(dim_h * 2),
            nn.ReLU(True),
            nn.Conv2d(dim_h * 2, dim_h * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(dim_h * 4),
            nn.ReLU(True),
            nn.Conv2d(dim_h * 4, dim_h * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(dim_h * 8),
            nn.ReLU(True),
        )
        
        self.fc_part = nn.Linear(dim_h * (2 ** 3), latent_dim)
        
    def forward(self, x):
        x = self.conv_part(x)
        x = x.squeeze()
        x = self.fc_part(x)
        return x

In [7]:
class WAE_DECODER(nn.Module):
    def __init__(self):
        super(WAE_DECODER, self).__init__()
        
        self.fc_part = nn.Sequential(
            nn.Linear(latent_dim, dim_h * 8 * 7 * 7),
            nn.ReLU(True),
        )
        
        self.conv_part = nn.Sequential(
            nn.ConvTranspose2d(dim_h * 8, dim_h * 4, 4),
            nn.BatchNorm2d(dim_h * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim_h * 4, dim_h * 2, 4),
            nn.BatchNorm2d(dim_h * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim_h * 2, 1, 4, stride=2),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.fc_part(x)
        x = x.view(-1, dim_h * 8, 7, 7)
        x = self.conv_part(x)
        return x

In [8]:
import torch.optim as optim

from torch.optim.lr_scheduler import StepLR


# models
wae_encoder, wae_decoder = WAE_ENCODER(), WAE_DECODER()
criterion = nn.MSELoss()

wae_encoder.train()
wae_decoder.train()

if torch.cuda.is_available():
    wae_encoder, wae_decoder = wae_encoder.cuda(), wae_decoder.cuda()

# this allows us to control descending/ascending gradient descent
one = torch.Tensor([1])
mone= one * -1

if torch.cuda.is_available():
    one, mone = one.cuda(), mone.cuda()
    
# optimizers
wae_enc_optim = optim.Adam(wae_encoder.parameters(), lr=start_learning_rate)
wae_dec_optim = optim.Adam(wae_decoder.parameters(), lr=start_learning_rate)

# schedulers
wae_enc_scheduler = StepLR(wae_enc_optim, step_size=30, gamma=0.5)
wae_dec_scheduler = StepLR(wae_dec_optim, step_size=30, gamma=0.5)

### MMD LOSS #1

In [9]:
def count_mmd_loss(X: torch.Tensor, Y: torch.Tensor, dim: int):
    batch_size = X.size(0)
    
    p2_norm_x = X.pow(2).sum(1).unsqueeze(0)
    norms_x = X.sum(1).unsqueeze(0)
    prods_x = torch.mm(norms_x, norms_x.t())
    dists_x = p2_norm_x + p2_norm_x.t() - 2 * prods_x
    
    p2_norm_y = Y.pow(2).sum(1).unsqueeze(0)
    norms_y = Y.sum(1).unsqueeze(0)
    prods_y = torch.mm(norms_y, norms_y.t())
    dists_y = p2_norm_y + p2_norm_y.t() - 2 * prods_y
    
    dot_prod = torch.mm(norms_x, norms_y.t())
    dists_c = p2_norm_x + p2_norm_y.t() - 2 * dot_prod
    
    stats = 0
    for scale in [.1, .2, .5, 1., 2., 5., 10.]:
        C = 2 * dim * 1.0 * scale
        res1 = C / (C + dists_x)
        res1 += C / (C + dists_y)
        
        res1 = (1 - torch.eye(batch_size).cuda()) * res1
        res1 = res1.sum() / (batch_size - 1)
        
        res2 = C / (C + dists_c)
        res2 = res2.sum() * 2. / batch_size

        stats += res1 - res2
    return stats

### MMD LOSS #2

In [10]:
class MMD_LOSS(nn.Module):
    def __init__(self, kernel_mul=2.0, kernel_num=5):
        super(MMD_LOSS, self).__init__()
        
        self.kernel_num = kernel_num
        self.kernel_mul = kernel_mul
        
    def guassian_kernel(self, source, target):
        n_samples = int(source.size()[0]) + int(target.size()[0])
        total = torch.cat([source, target], dim=0)
        
        total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        L2_dist = ((total0 - total1) ** 2).sum(2)
        bandwidth = torch.sum(L2_dist.data) / (n_samples ** 2 - n_samples)
        bandwidth /= self.kernel_mul ** (self.kernel_num // 2)
        bandwidth_list = [bandwidth * (self.kernel_mul ** i) for i in range(self.kernel_num)]
        kernel_val = [torch.exp(-L2_dist / bandwidth_temp) for bandwidth_temp in bandwidth_list]
        return sum(kernel_val)
    
    def forward(self, source, target):
        batch_size = int(source.size()[0])
        kernels = self.guassian_kernel(source, target)
        
        XX = kernels[:batch_size, :batch_size]
        YY = kernels[batch_size:, batch_size:]
        XY = kernels[:batch_size, batch_size:]
        YX = kernels[batch_size:, :batch_size]
        
        return torch.mean(XX + YY - XY - YX)

### Training

In [11]:
import os

from torchvision.utils import save_image


mmd_loss_model = MMD_LOSS()
if torch.cuda.is_available():
    mmd_loss_model = mmd_loss_model.cuda()

for epoch in range(epochs_num):
    step = 0
    for (images, _) in tqdm(train_loader):
        if torch.cuda.is_available():
            images = images.cuda()
            
        wae_enc_optim.zero_grad()
        wae_dec_optim.zero_grad()
        
        # DECODER TRAINING
        z = wae_encoder(images)
        x_recon = wae_decoder(z)
        if torch.cuda.is_available():
            x_recon = x_recon.cuda()
        
        recon_loss = criterion(x_recon, images)
        
        # MMD LOSS
        z_fake = Variable(torch.randn(batch_size, latent_dim) * sigma_coeff)
        if torch.cuda.is_available():
            z_fake = z_fake.cuda()
            
        z_real = wae_encoder(images)
        
        mmd_loss = count_mmd_loss(z_real, z_fake, latent_dim)
        mmd_loss = mmd_loss.mean()
        # mmd_loss = mmd_loss_model(z_real, z_fake)
        
        total_loss = recon_loss - mmd_loss
        total_loss.backward()
        
        wae_enc_optim.step()
        wae_dec_optim.step()
        
        step += 1
        
        if (step + 1) % 100 == 0:
            print("Epoch: [%d/%d], Step: [%d/%d], Reconstruction Loss: %.4f" %
                 (epoch + 1, epochs_num, step + 1, len(train_loader), recon_loss.data.item()))
        
    if (epoch + 1) % 1 == 0:
        test_iter = iter(test_loader)
        test_data = next(test_iter)
        
        z_real = wae_encoder(Variable(test_data[0]).cuda())
        reconst = wae_decoder(torch.randn_like(z_real)).cpu().view(batch_size, 1, 28, 28)
        
        if not os.path.isdir('./data/reconst_images2'):
            os.makedirs('./data/reconst_images2')
            
        save_image(test_data[0].view(-1, 1, 28, 28), './data/reconst_images2/wae_mmd_input.png')
        save_image(reconst.data, './data/reconst_images2/wae_mmd_images_%d.png' % (epoch + 1))

 17%|████████████████████▏                                                                                                   | 101/600 [00:08<00:39, 12.65it/s]

Epoch: [1/50], Step: [100/600], Reconstruction Loss: 0.0794


 34%|████████████████████████████████████████▏                                                                               | 201/600 [00:16<00:31, 12.56it/s]

Epoch: [1/50], Step: [200/600], Reconstruction Loss: 0.0650


 50%|████████████████████████████████████████████████████████████▏                                                           | 301/600 [00:24<00:23, 12.58it/s]

...

Epoch: [50/50], Step: [400/600], Reconstruction Loss: nan


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████                    | 500/600 [00:39<00:07, 12.58it/s]

Epoch: [50/50], Step: [500/600], Reconstruction Loss: nan


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:47<00:00, 12.59it/s]


Epoch: [50/50], Step: [600/600], Reconstruction Loss: nan
